In [1]:
%%javascript
$('#appmode-leave').hide();                          // Hides the edit app button.
$('#appmode-busy').hide();                           // Hides the kernel busy indicator.

<IPython.core.display.Javascript object>

In [2]:
import os
import ipywidgets as ipw
from glob import glob
import json
import pandas as pd
import numpy as np
from PIL import Image
from itertools import cycle
from io import BytesIO
from time import time
USERNAME=os.environ.get('APPMODE_USER', 'Not logged in')
HOSTNAME=os.environ.get('HOSTNAME', 'anon')
BUTTONS_PER_ROW = 3
BUTTON_WIDTH = "150px"
UNKNOWN_OPTION = 'Unknown'

/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
with open('task.json', 'r') as f:
    annotation_task = json.load(f)
    annot_df = pd.DataFrame(annotation_task['dataset']['dataframe'])
print('Loaded annotation task', annot_df.shape[0], 'images')
button_ids = annot_df[annotation_task['dataset']['output_labels']].unique().tolist()
if UNKNOWN_OPTION is not None:
    button_ids += [UNKNOWN_OPTION]

image_path_dict = {c_path: os.path.join(annotation_task['dataset']['base_image_directory'], 
                                    c_path)
                                    for c_path in annot_df[annotation_task['dataset']['image_path']]}
image_keys = list(image_path_dict.keys())

Loaded annotation task 110 images


In [4]:
global out_rows
out_rows = []
def submit_label(label_id, label_name):
    """
    should be linked to some kind of database
    """
    global out_rows
    print('Submitting', label_name, 'for', label_id, 'from', USERNAME, 'part of', HOSTNAME, '@', time())
    out_rows += [dict(label_name=label_name, label_id=label_id, user=USERNAME, session=HOSTNAME, time=time())]
    return pd.DataFrame(out_rows).to_html()

In [5]:
title_box = ipw.HTML(value=f'<h1> Welcome {USERNAME}</h1><h2> Select the most appropriate label for the given image')
cur_image_view = ipw.Image(layout=ipw.Layout(width="512px"),  
                  disabled=True)
cur_img_id = ipw.Text(layout=ipw.Layout(width="64px"),  
                  disabled=True)
annot_results = ipw.HTML(value='')

SESSION_ID = None
np.random.shuffle(image_keys)
image_iter = cycle(iter(image_keys))

def get_next_image():
    c_key = next(image_iter)
    cur_img_id.value = c_key
    c_img = Image.open(image_path_dict[c_key])
    bio_obj = BytesIO()
    c_img.save(bio_obj, format='png')
    bio_obj.seek(0)
    cur_image_view.value=bio_obj.read()

def on_click(btn):
    if btn is not None:
        annot_results.value = submit_label(cur_img_id.value, btn.description)
    get_next_image()
        

def mk_btn(description):
    btn = ipw.Button(description=description, layout=ipw.Layout(width=BUTTON_WIDTH))
    btn.on_click(on_click)
    return btn

rows = []
c_row = []
for i, but_name in enumerate(button_ids, 1):
    c_row+=[mk_btn(but_name)]
    if (i % BUTTONS_PER_ROW)==0:
        rows+=[ipw.HBox(c_row)]
        c_row=[]
rows+=[ipw.HBox(c_row)]
on_click(None) # initialize
ipw.VBox((title_box, cur_image_view)+tuple(rows)+(annot_results,))

VBox(children=(HTML(value='<h1> Welcome mr_head</h1><h2> Select the most appropriate label for the given image'), Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x00\x00\x00\x02\x00\x08\x02\x00\x00\x00{\x1aC\xad\x00\x01\x00\x00IDATx\x9c\x84\xbd=\x8fm\xcbv\x97_Usv\xef\xee\xde\xbd\xcf\xb9\xd7\x18\xb0\x08\x80\x00\xc9\x02\xcb\x02\t\x89\x10\xc8\x1c \xe3\xd4\x08\x01\x11B\xe2\x1b\x10\xf0\x05\xf8\x00$d\x84\x84\x8e\x10\t\x16r\x80\x1c ,\x13\x80\x84dd!\xac\x0b\xf7\xdes\xcf~\xe9\xd75\xab\xfe\xc18\xe39O\xd5\xec\xe3\xff\n\xb6\xd6^=_\xaaF\x8d\xf1\x1b\xaf5\xaa\xfe\xdd\xbf\xfbw\x9f\xf59\x8e\xe3\xf5\xf5\xf58\x8e\xe38\xc6\x18\xf1o)e\x8cQk\x8d/\xad\xb5\xf8\xb1\x94Rk\xad\xb5\xf2\xa3\xff\xca\xf5\xf1%.+\xa5,\xb7\xfb\x1a>q1?rY\xfc\xc8\xa3\x96\'s\xbd/\x1bcl\xdb\x16\xbf\xf4\xde\xe3wF~\x1cGk\x8d\xe7\x9f\x9f\xc6H<\x11\x8f\xd9\xf3=\x8ec\xdb\xb6\xf3\x18|\xe32MH\xdaZ\xf3\xd3z\xefc\x8c\x18p|\x81\x02q\xfd\xf5\xf5\xf5\xbe\xef\xdb\xb6\xed\xfb~}}\xdd\xf2Sk\x8dG\xc5\xd3\x18\t\xcf\x8f\x97\xee\xfb\x1e?\xd6Z\xef\xee\xee\xf6}\x8f\xef\xf1\xc0Z\xeb\xf5\xf5u\x90\xae\xd6zuu\xb5m\xdb\xf5\xf55\x03\xe3OA@^\xd1Z\x8b\xa1\xf6\xde\xf7}\x8f+\x17\xaarq\xcc\x8e!1\xaa\x98\xe0q\x1c,\x16K\x10\x17\x07AZk\xb1\xb2\xb0h\x0c\xe9r\xb9\xc4\x18\x8e\xe3\x80\x0e1\xc8m\xdb N)e\xdb\xb6\xe0\xf0\xf8\xc2\xf21\xbc\xb87\xae\x89\xe9\xc4\x8f1A\xf3\xa7\x85%\x9e\x16C]\x18,\xde\x15c[\xder\xb9\\\xb6m\xeb\xbd\xc7\xa4\xb8\x80\tB\xb1\xe0\xe4\x85E____^^.\x97\x0b\xe3?\x8eca\x1b\x0b\x11\xaceAf`\x97\xcbe\x8cq\xb9\\\x02\r^__c^\xe6\xb1\xe0\x16\x18\x18\xceg\x00\xcc:\xd6%^\x17s\x8c\xe1\xb5\xd6\x82\xaa\xb1(\xfb\xbe\xfbOg\x10\x88\xcb\xb6m\xbb\\.\xf0\x1bSX\xa0\x03\xb6\\.\x80\x92P\x1b\xb2\xc0\x1b\x0b\x02\xc4\xa8\xe2\xbf\x0b\x86\xbc\xf9d^\x1a\xaf(o\xc1\xe6r#o\xbc\\.\xfb\xbe\xf3\xb4\x85\xc7\x98T\x91L\x05\xb7\xf0\x1c\xb84\x18\x92\x01\x04?\xf8\xae1\xc6\xfe\xf0\xf0\xf0\xf4\xf4\xf4\xf4\xf4\x04\x03].\x97\xb8\x94\x9b\xe3{9\xe1\xb2\xa7\x04+/\xa4Y\xb0\x18H\xf25\x0b:,42p#u\x0b\n/0m\xa4X\xa8l\n\xfa\xc3\xea\xa2-\xe2\xbf1\xe03~\xbd9\xd3e:\x90ey\x08w\xc1.,6\x17,B\xcb\xe0\xaf\xaf\xaf\xaf\xaf\xaf\xdf\xbd{\x07\xfc\xc58c\xe4!\xff%\xe59\xde\x1b\x03@*Bn___\x11\xcb}\xdfC\xe4\xb6m{yy\t\x8d\x122VJ\x89\x8b\xf7}\x8f_\xe0\xa1\x18\x0f\xf8e\xb2\x87\x0c???\x03\x82o.\x93\x11\xa1\xe6\'@!\x18\xda\x9cf\xb6\x89[x\x1d\x80\x18\x97\x05x\xf1\t\x95\x16\x02\x10\x10\x0f\x9da\xf20}\xe2\xc9\x01|\xa1\x8a\xe0\x871\xc6\xd3\xd3\x93\xc5\xf8\xccZ\xbdw\x94\xd6\xc2\x15\\|uu\x15\xc3\xb3\x1e2W\x03\xd0\xbc\x0b\x95\xc3s\x82\xbc^\xe8R\xca\xf3\xf3\xf3\xeb\xeb\xeb\xc3\xc3\x03km\x11\xe6\xbf\xf1\x84\x80\x06\xcb\x14\xec\x11\x9c\x802\x8b\xe1\xc5j\x06>\xc4\xf5AFsr\xac\xc5\xd9\xc6b\xa1\x17\x1eX \xde\xb7\xb04\xbe=\xd4\x83%\xf7\xfc\xa8!\xb3\xcc\xe4Z\xf8-\x9e\xcfe5M7\xff\x95?A%T\x97q\x0f\xb8\xe0\xf96\xbfx2\x8a\x9c\x87\x98,#\x8dT+Tf\x1a\x7fb<<\xcd\xac\xb8\xa0V\x995\xfdB1X\xae\xb5\xb6_.\x97\x97\x97\x97\xd7\xd7\xd7\xd7\xd7\xd7K~b\xc9\xc7lx\x96\x93\xa9\xbe\xe84\xc3\xfa\x90O\xc0T\xb9\xf2\xfc(\xfed@D\x9c\x16\xf5\xf0\xe6\x8d\xcbH\xcc\xdc\xbc\xd1\x03x\xf3\x81\xdch\n\xbeiex\x16cVl\x8b.4\x05\xfcK\x93\xcfd\x92\xc6\xed1rP\xe9\xfa\xfa\xfa\xea\xea*,q\x0c\x04\x9e\x16\xb8\xff\xfa\xfa\x8a\x9dk\xb6`\x00\xe8\x9b06\xe3\xfaZ\xeb\xbe\xefWWW\xd8\xd4\xbdw\x100\xc6p\xb9\\\xc2\xfc\x07w,?#=\x18H\x01\x05XS\xab\xc0e\xe9-<1\xf2\x98\x0e\xff\x8d1\xc0\xcd1)L\x93"\xec\x08\xfa\xbf\xbc\xbc\x04\x01\xe3\t\x16\xaa\xa0\x15\xafC+0\x92\x97\x97\x97\xa096o|\xa9\xf2\xc3L\x01\xe6\x82\xcdd\xde0\xfd\x83\xf2OOOE\x90\xc4\xecj\xeaZ\x86g\xab\x8b\xd1\xf2X+\xc8\x80\x98\xe7\xe7\xe7\xc7\xc7G?\x9fW\xf3\xfc\x90\xeb\x92\xea\xca\xe3\x1f\xb2<l\xb5\xc4\x95\xf1a\x89C\x19\xc0\xedEH\xca\xf4\r\x88\xcbr[\xe2\x16,3\x87,"\xb3\x80\xa6\xc7\x83\xde2k-7\x0e\x19\x7f\x96z(S\xe5\xd0\x18=\x18\x9b\x07\xc9[\xec\xc5\xd6Y\x01x\xb4~\x17\xcf\xf4\x80\x8d\xe0&#\x84]\x84\x05\xea\xf1c\x08/\xcf\x19\xe9x\x95R\xf6}\x7f\xf7\xee\x1d\x13\xb7\xaa\xd8\xbd\xf0|l,\x80\xc8\xbe\xb2\x08\x1f\xbdl\xbe\x00\x115\x07\xf3\xbd\xe8Sg\x1b\x1f\xfe\xf3\xbd\xe6N\x0ffy\xce\x10\xca//\xf2h\x7f\xe8F\xbeX{-\xd7\x0ci\xe37/0K\xd5\x13\xee/\x13ys\xa85M\x9e\xab\xab\xab0\xf9kj\xfe\x9aQ\x85\x92p

Submitting Atelectasis for 00028676_000.png from mr_head part of jupyter-chestrays-2djupyanno-2dg0orrzh8 @ 1533218420.094979
Submitting Effusion for 00017747_029.png from mr_head part of jupyter-chestrays-2djupyanno-2dg0orrzh8 @ 1533218420.8936095
Submitting No Finding for 00013993_071.png from mr_head part of jupyter-chestrays-2djupyanno-2dg0orrzh8 @ 1533218421.868014
Submitting Pneumonia for 00028180_005.png from mr_head part of jupyter-chestrays-2djupyanno-2dg0orrzh8 @ 1533218423.9775312
Submitting Pleural_Thickening for 00011831_006.png from mr_head part of jupyter-chestrays-2djupyanno-2dg0orrzh8 @ 1533218424.747326
Submitting Effusion for 00010936_025.png from mr_head part of jupyter-chestrays-2djupyanno-2dg0orrzh8 @ 1533218425.5518644
Submitting Edema for 00010805_042.png from mr_head part of jupyter-chestrays-2djupyanno-2dg0orrzh8 @ 1533218426.5862942
